# BlockSci Overview

This tutorial will show you how to use the BlockSci python interface.

## General setup

You'll need to execute these lines first in order to initialize BlockSci.

To do so, enter the cell by clicking on it, then press *Shift+Enter* (or select it and go to *Cell->Run Cells* in the toolbar at the top).

In [ ]:
import blocksci
import pandas as pd
import matplotlib as plt
%matplotlib notebook

In [ ]:
chain = blocksci.Blockchain("/blocksci/bitcoin")

## Blocks in the Blockchain
The `chain` object represents a blockchain. It contains all blocks in the blockchain.

In [ ]:
len(chain)

You can access each block directly by its index (i.e. its height).

In [ ]:
chain[400000]

Block 400000 was created on

In [ ]:
chain[400000].time

It contains 1660 transactions (including the coinbase transaction):

In [ ]:
len(chain[400000])

You can access blocks at the end of the blockchain using negative indices. For example, `chain[-1]` returns the last block in the chain.

In [ ]:
chain[-1]

In [ ]:
chain[-1].time

Here are a few more examples for selecting blocks in the chain:

In [ ]:
chain[0]          # selects the first block in the chain
chain[1]          # selects the second block
chain[1:3]        # selects the second to fourth block
chain[1:]         # selects all blocks after the first
chain[:5]         # selects the first five blocks
chain[:-2]        # selects all but the last two blocks

## Transactions
You can access transactions in a block like you a block in the chain (by index or by iterating over the block).

In [ ]:
myblock = chain[400000]
myblock[0]

The first transaction in every block is the coinbase transaction.

In [ ]:
myblock[0].is_coinbase

We can choose other transactions by their position in the block. If you just print the transaction, it will give you helpful output about the contents of the transaction. This transaction has three inputs, two outputs and a size of 520 bytes.

In [ ]:
mytx = myblock[1234]
mytx

This transaction paid 0.0001 BTC in transaction fees (amounts are denoted in satoshi, 1 BTC equals 100,000,000 satoshi).

In [ ]:
mytx.fee / 1e8

## Inputs and Outputs

Every transaction has a list of inputs and outputs

In [ ]:
for i in mytx.inputs:
    print(i)

In [ ]:
for o in mytx.outputs:
    print(o)

Every input and output is associated with an address.

In [ ]:
myaddr = mytx.outputs[0].address
myaddr

You can see all the transactions that an address appears in

In [ ]:
myaddr.txes()

## Creating a chart

Pandas is a python library that provides advanced data structures for data analysis.

First, we will create a time series (`Series`) of the Bitcoin block size (in KB). A time series consists of a list of values (the blocksize) and an index of corresponding length (the timestamp of the block). We will then average the data over a time frame of 1 month and plot the results.

In [ ]:
block_times = [block.time for block in chain]
block_sizes = [block.size_bytes / 1000 for block in chain]
ds_block_size = pd.Series(block_sizes, index=block_times)
size_plot = ds_block_size.resample("1 M").mean().plot()
size_plot.set_ylabel("Block size (KB)")

Next, we will create a plot with two time series. For this we make use of the `DataFrame`.

In [ ]:
tx_per_block = [len(block) for block in chain]
df_blocksize = pd.DataFrame({
    "Block size (KB)": block_sizes,
    "Number of transactions": tx_per_block
}, index=block_times)
df_blocksize.resample("1 M").mean().plot()

## Satoshi vs. BTC

Values in Bitcoin (and BlockSci) are denoted in Satoshi, with 1 BTC = 100,000,000 Satoshi. You can convert from Satoshi to Bitcoin and vice versa as follows:

In [ ]:
# The total output value of the transaction in Satoshi
mytx.output_value

In [ ]:
# The same in BTC
COIN = 1e8
mytx.output_value / COIN

## Currency Conversion

BlockSci comes with a built-in class that provides currency conversion (using the CoinDesk price API):

In [ ]:
converter = blocksci.CurrencyConverter() # you can pass the currency as string, default is USD

In [ ]:
converter.bitcoin_to_currency(date='2017-06-30', value=1)

In [ ]:
converter.currency_to_bitcoin(date='2017-06-30', value=1000)